### TwitterBot (tweet) that reads text from a file and tweets automatically

In [ ]:
import tweepy
from credentials import *
from time import sleep

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

path = 'C:/Users/Spurthi/OneDrive - University of Illinois at Chicago/Academics/3. Spring Courses/1. IDS 576 - Predictive/Assign2/Q2/text_corpus.txt'
text = open(path,'r')

In [ ]:

file_lines = text.readlines()
text.close()

def tweet():
    for line in file_lines:
        try:
            print(line)
        # Add if statement to ensure that blank lines are skipped
            if line != '\n':
                api.update_status(line)
                sleep(360)
        # Add an else statement with pass to conclude the conditional statement
            else:
                pass

        except tweepy.TweepError as e:
            print (e.reason)
        # Add sleep method to space tweets by 5 seconds each
        sleep(900)
        
tweet()
api.update_status(line)

### Training RNN (LSTM)  and tweet using TwitterBot above

In [8]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [9]:
import itertools

In [25]:
path = 'C:/Users/Spurthi/OneDrive - University of Illinois at Chicago/Academics/3. Spring Courses/1. IDS 576 - Predictive/Assign2/Q2/text_corpus.txt'
text = open(path,'r')
file_lines= text.readlines()
text.close()

In [26]:
# create mapping of unique chars to integers
chars = sorted(list(set(file_lines)))
char_to_int = dict((c, i) for i, c in enumerate(chars))


In [27]:
n_chars = len(file_lines)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  40000
Total Vocab:  25722


In [31]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 140
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = file_lines[i:i + seq_length]
    seq_out = file_lines[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)


Total Patterns:  39860


In [38]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)


In [39]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:
class RNN:
    def __init__(self,len_h,len_x):
        self.h = numpy.zeros(len_h)
        self.w = numpy.random.randn(len_h,len_h+len_x)
        self.bias = numpy.random.randn(len_h)
    def step (self, x_t):
        activation = numpy.dot(self.w, numpy.hstack((self.h,x_t)))+self.bias
        self.h =numpy.tanh(activation)
        return self.h
    
rnn = RNN(3,4)
for i in range (5):
    x_t = numpy.random.randn(4)
    h_t = rnn.step(x_t)
    print (h_t)

[ 0.35695206  0.16913434  0.00246825]
[-0.33323064 -0.64385899 -0.88633615]
[ 0.49771852 -0.09338396  0.99697232]
[-0.49421432  0.92865159 -0.99427073]
[-0.94363523 -0.85865197 -0.68915677]


In [41]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [43]:
model.fit(X, y, nb_epoch=1, batch_size=128, callbacks=callbacks_list)
"""start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''   
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(10):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()"""

C:\Users\Spurthi\Anaconda2\envs\py35\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
39860/39860 [==============================] - 1088s - loss: 9.7167  


'start_index = random.randint(0, len(text) - maxlen - 1)\n\nfor diversity in [0.2, 0.5, 1.0, 1.2]:\n        print()\n        print(\'----- diversity:\', diversity)\n\n        generated = \'\'   \n        sentence = text[start_index: start_index + maxlen]\n        generated += sentence\n        print(\'----- Generating with seed: "\' + sentence + \'"\')\n        sys.stdout.write(generated)\n\n        for i in range(10):\n            x = np.zeros((1, maxlen, len(chars)))\n            for t, char in enumerate(sentence):\n                x[0, t, char_indices[char]] = 1.\n\n            preds = model.predict(x, verbose=0)[0]\n            next_index = sample(preds, diversity)\n            next_char = indices_char[next_index]\n\n            generated += next_char\n            sentence = sentence[1:] + next_char\n\n            sys.stdout.write(next_char)\n            sys.stdout.flush()\n        print()'

In [45]:
# load the network weights
filename = "weights-improvement-00-9.7167.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [46]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [47]:
# pick a random input pattern as a seed sequence, and print generated characters 
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")

print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    #sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")



Seed:
" Good news, good news; the ladies have prevail'd,
The Volscians are dislodged, and Marcius gone:
A merrier day did never yet greet Rome,
No, not the expulsion of the Tarquins.

SICINIUS:
Friend,
Art thou certain this is true? is it most certain?

Second Messenger:
As certain as I know the sun is fire:
Where have you lurk'd, that you make doubt of it?
Ne'er through an arch so hurried the blown tide,
As the recomforted through the gates. Why, hark you!
The trumpets, sackbuts, psalteries and fifes,
Tabours and cymbals and the shouting Romans,
Make the sun dance. Hark you!

MENENIUS:
This is good news:
I will go meet the ladies. This Volumnia
Is worth of consuls, senators, patricians,
A city full; of tribunes, such as you,
A sea and land full. You have pray'd well to-day:
This morning for ten thousand of your throats
I'd not have given a doit. Hark, how they joy!

SICINIUS:
First, the gods bless you for your tidings; next,
Accept my thankfulness.

Second Messenger:
Sir, we have all


In [ ]:
new_path= 'C:/Users/Spurthi/OneDrive - University of Illinois at Chicago/Academics/3. Spring Courses/1. IDS 576 - Predictive/Assign2/Q2/Trained.txt'
new_text_corpus =open(new_path,'w')
for value in pattern:
    #print (int_to_char[value])
    new_text_corpus.write(int_to_char[value])
#print(title)

In [22]:
new_text_corpus.close()

In [1]:
import tweepy
from credentials import *
from time import sleep

path = 'C:/Users/Spurthi/OneDrive - University of Illinois at Chicago/Academics/3. Spring Courses/1. IDS 576 - Predictive/Assign2/Q2/Trained.txt'
text = open(path,'r')

file_lines = text.readlines()
text.close()


In [ ]:
tweet()
api.update_status(line)

Third Conspirator:

Sir, his stoutness

When he did stand for consul, which he lost

By lack of stooping,--



AUFIDIUS:

That I would have spoke of:

Being banish'd for't, he came unto my hearth;

